In [ ]:
!pip install textblob 
#language recognition

In [12]:
import requests
import csv
import pandas as pd
import numpy as np
import datetime #today's date
from textblob import TextBlob 
from bs4 import BeautifulSoup
import re
import time #delay for scrapping multiple articles

In [13]:
url = "https://www.myrhline.com/actualite-rh/de-la-gpec-et-au-workforce-planning-les-5-evolutions-a-connaitre.html"

In [14]:
df= pd.DataFrame(columns=['art_content','art_content_html','art_extract_datetime','art_lang','art_title','art_url','src_name','src_type','src_url','src_img','art_auth','art_tag']  )
df

,art_content,art_content_html,art_extract_datetime,art_lang,art_title,art_url,src_name,src_type,src_url,src_img,art_auth,art_tag


In [15]:
""" 
In: url
Out: a tuple of strings
"""

def get_all(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    art_content_html = soup.find("div", {"class": "post-detail-wrap"})
    if art_content_html is None:
        art_content_html = no_data

    art_content = str()
    for paragraph in art_content_html:
        art_paraph = paragraph.text.replace('\xa0', '')
        art_content += art_paraph
    if art_content is None:
        art_content = no_data    
   
    if soup.find("meta", {"property": "article:modified_time"}) is not None:
        art_extract_datetime = soup.find("meta", {"property": "article:modified_time"})["content"]
    elif soup.find("meta", {"property": "article:published_time"}) is not None:
        art_extract_datetime = soup.find("meta", {"property": "article:published_time"})["content"]
    else:
        art_extract_datetime = datetime.datetime.now()
    
    if soup.find("meta", {"property": "og:locale"}) is not None:
        art_lang = soup.find("meta", {"property": "og:locale"})["content"]
    else:
        art_lang = no_data
    
    if soup.find("meta", {"property": "og:title"}) is not None:
        art_title = soup.find("meta", {"property": "og:title"})["content"]
    else:
        art_title = no_data
        
    if soup.find("meta", {"property": "og:url"}) is not None:
        art_url = soup.find("meta", {"property": "og:url"})["content"]
    else:
        art_url = no_data
    
    if soup.find("meta", {"property": "og:site_name"}) is not None:
        src_name = soup.find("meta", {"property": "og:site_name"})["content"]
    else:
        src_name = no_data
    
    src_type = "xpath_source" #default value  
    
    if soup.find("div", {"class": "cscra-brand dhve-mobile-logo dhve-retina-logo hav-normal-logo"}).find("a") is not None:
        src_url = soup.find("div", {"class": "cscra-brand dhve-mobile-logo dhve-retina-logo hav-normal-logo"}).find("a")["href"]
    else:
        src_url = no_data
    
    if soup.find("meta", {"property": "og:image"}) is not None:
        src_img = soup.find("meta", {"property": "og:image"})["content"]
    else:
        src_img = no_data
        
    if soup.find("meta", {"name": "author"}) is not None:   
        art_auth = soup.find("meta", {"name": "author"})["content"]
    else:
        art_auth = no_data

    if soup.find_all("a", {"rel": "tag"}) is not None:
        art_tags = soup.find_all("a", {"rel": "tag"})
        art_tag = []
        for tag in art_tags:
            art_tag.append(str(tag.text))
    else:
        art_tag = no_data
    
    
    return art_content, art_content_html, art_extract_datetime, art_lang, art_title, art_url,\
        src_name, src_type, src_url, src_img, art_auth, art_tag

'art_content' [0],'art_content_html'[1],'art_extract_datetime'[2],'art_lang'[3],
    'art_title'[4],'art_url'[5],'src_name'[6],'src_type'[7],'src_url'[8],'src_img'[9],'art_auth'[10],'art_tag'[11]
    

In [16]:
"""
In: url
Out: url of the previous article
"""

def get_prev(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    prev_soup = soup.find("div", {"class": "col-sm-6 col-xs-6"}).find("a")
    if prev_soup is not None:
        prev_url = prev_soup["href"]
        return prev_url
    else:
        return None
    
"""
In: url
Out: url of the following article
"""
    
def get_next(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    next_soup = soup.find("div", {"class": "col-sm-6 col-xs-6 text-right"}).find("a")
    if next_soup is not None:
        next_url = next_soup["href"]
        return next_url
    else:
        return None


In [17]:
"""
In: url 
Out: 
"""

def art_date(url):
    url = get_prev(url)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    if soup.find("meta", {"property": "article:modified_time"}) is not None:
        art_extract_datetime = soup.find("meta", {"property": "article:modified_time"})["content"]
    elif soup.find("meta", {"property": "article:published_time"}) is not None:
        art_extract_datetime = soup.find("meta", {"property": "article:published_time"})["content"]

    date_article = datetime.datetime.strptime(art_extract_datetime, '%Y-%m-%dT%H:%M:%S+00:00')
    year = int(date_article.strftime('%Y'))
    return year    

In [18]:
"""
In: url, year after which the articles have to be published
Out: list of preceding articles
"""

def all_before(url, year):
    before = []
    while get_prev(url) is not None:
        url = get_prev(url)
        published = art_date(url)
        if published > year:
            before.append(url)
                
    return list(set(before))

"""
In: url, year after which the articles have to be published
Out: list of following articles
"""

def all_after(url, year):
    after = []
    while get_next(url) is not None:
        url = get_next(url)
        published = art_date(url)
        if published > year:
            after.append(url)
    return list(set(after))

In [19]:
"""
In: url, year after which the articles have to be published
Out: list of all articles
"""

def all_in(url, year):
    all_be = all_before(url, year)
    all_af = all_after(url, year)
    url_date = art_date(url)
    if url_date > year:
        return list(set(all_be + [url] + all_af))

In [20]:
"""
In: list of urls
Out: dataframe with all scraped articles
"""

def scrap_all(list_urls):
    i = 1
    df= pd.DataFrame(columns=['art_content','art_content_html','art_extract_datetime',\
                              'art_lang','art_title','art_url','src_name','src_type','src_url',\
                              'src_img','art_auth','art_tag'])
    for url in list_urls:
        df.loc[i] = get_all(url)
        i+=1
    return df  

In [21]:
"""
In: url of a random article
Out: dataframe with all scraped articles
"""

def do_it_all(url):
    list_urls = all_in(url)
    scrap_all(list_urls)

In [ ]:
do_it_all(url)